# HW8

# II. Analyzing Data

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = "{:,.4f}".format

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from arch import arch_model

import warnings
warnings.filterwarnings("ignore")

import importlib
import utils as ut
importlib.reload(ut)

<module 'utils' from '/Users/chuan/github/0_Quantitative_Portfolio_Manager/homework/utils.py'>

# 2. LTCM Risk Decomposition

- On Canvas, find the data file, **“ltcm exhibits data.xlsx”**. Get the gross and net (total) returns of LTCM from **“Exhibit 2”**.
- Get the returns on SPY as well as the risk-free rate from the file, **“gmo analysis data”**.


In [2]:
file_path = './../data/ltcm_exhibits_data.xlsx'

returns = pd.read_excel(file_path, sheet_name='Exhibit 2', skiprows=3, usecols=[0, 2, 3]).iloc[:-4]
returns.columns = ['Date', 'Gross Return', 'Net Return']
returns.set_index('Date', inplace=True)
returns.index = pd.to_datetime(returns.index) + pd.tseries.offsets.MonthEnd(0) # align index to end of month
display(returns.head())

file_path = './../data/gmo_analysis_data.xlsx'

risk_free_rate = pd.read_excel(file_path, sheet_name='risk-free rate', index_col=0)
risk_free_rate.index.name = 'Date'
risk_free_rate.index = pd.to_datetime(risk_free_rate.index)
risk_free_rate = risk_free_rate.loc[returns.index]
display(risk_free_rate.head())

spy = pd.read_excel(file_path, sheet_name='returns (total)', index_col=0)[['SPY']]
spy.index.name = 'Date'
spy = spy.loc[returns.index]
excess_spy = spy.sub(risk_free_rate['US3M'], axis=0)
display(excess_spy.head())

excess_returns = returns.sub(risk_free_rate['US3M'], axis=0)
excess_returns.head(10)

,Gross Return,Net Return
Date,,
1994-03-31,-0.0110,-0.0130
1994-04-30,0.0140,0.0080
1994-05-31,0.0680,0.0530
1994-06-30,-0.0390,-0.0290
1994-07-31,0.1160,0.0840


,US3M
Date,
1994-03-31,0.0030
1994-04-30,0.0033
1994-05-31,0.0036
1994-06-30,0.0036
1994-07-31,0.0037


,SPY
Date,
1994-03-31,-0.0449
1994-04-30,0.0079
1994-05-31,0.0123
1994-06-30,-0.0264
1994-07-31,0.0287


,Gross Return,Net Return
Date,,
1994-03-31,-0.0140,-0.0160
1994-04-30,0.0107,0.0047
1994-05-31,0.0644,0.0494
1994-06-30,-0.0425,-0.0326
1994-07-31,0.1123,0.0803
1994-08-31,0.0341,0.0261
1994-09-30,-0.0080,-0.0070
1994-10-31,0.0057,-0.0003
1994-11-30,0.0722,0.0562


## 2.1 Summary stats.
### (a) For both the gross and net series of LTCM excess returns:
Report the **mean**, **volatility**, and **Sharpe ratios** (annualized).

### (b) Report the **skewness**, **kurtosis**, and **(historic) VaR(0.05)**.


In [3]:
ut.summary_statistics_annualized(excess_returns)

,Mean,Vol,Sharpe,Min,Max,Skewness,Kurtosis,VaR 5%,CVaR 5%,Max Drawdown,Bottom,Peak,Recovery,Duration
Gross Return,0.2421,0.1362,1.7769,-0.1053,0.1123,-0.2877,1.5866,-0.0304,-0.0730,-0.1689,1998-06-30,1998-04-30,None,None
Net Return,0.1554,0.1118,1.3901,-0.1053,0.0803,-0.8102,2.9269,-0.0264,-0.0687,-0.1761,1998-07-31,1997-12-31,None,None


In [4]:
ut.summary_statistics_annualized(excess_spy)

,Mean,Vol,Sharpe,Min,Max,Skewness,Kurtosis,VaR 5%,CVaR 5%,Max Drawdown,Bottom,Peak,Recovery,Duration
SPY,0.1738,0.1123,1.5479,-0.0562,0.0749,-0.4335,-0.3620,-0.0464,-0.0514,-0.0562,1997-08-31,1997-07-31,1997-12-31,153 days



### (c) Comment on how these stats compare to SPY and other assets we have seen.
How much do they differ between gross and net?

- Comparing the Net Monthly Performance of LTCM and Excess returns of SPY, LTCM displays a higher return, with a very similar volatility to SPY. Thus the sharpe ratio of LTCM net of fee and other charges is slightly higher than SPY's.
- The excess net returns of LTCM however, underperform SPY with similar volatility levels and thus have a slightly lower sharpe ratio.
- However, looking at other moments, LTCM Net returns are more negatively skewed and have a significantly fatter tail compared to SPY, indicating the presence of heavy negative monthly returns over the sample period. Although, since the VaR of LTCM is lower compared to SPY, the indication is that these negative returns are fewer in frequency.

## 2.2 Regression Analysis
Using the series of net LTCM excess returns $ \tilde{r}_{LTCM,t} $, estimate the following regression:
$$
\tilde{r}_{LTCM,t} = \alpha + \beta_m \tilde{r}_{m,t} + \epsilon_t
$$

### (a) Report $ \alpha $, $ \beta_m $, and the $ R^2 $ stat.

### (b) From this regression, does LTCM appear to be a “closet indexer”?

### (c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?

In [8]:
ut.time_series_regression_annualized(excess_returns, excess_spy, intercept=True, annual_factor=12)['df']

,Alpha,SPY Beta,Treynor Ratio,IR,R^2
Gross Return,0.2108,0.1798,1.3460,1.5648,0.0220
Net Return,0.1315,0.1371,1.1331,1.1881,0.0190


- b). Definitely not. Low $\beta$ and low $R^2$, as well as high alpha mean that the fund is not tracking the market and is in no way a closet indexer. This is likely because they are trading fixed income primarily, and most of their trades are pairs/arbitrage trades, meaning that by definition they don't have exposure to the market.
- c). Very high net alpha at 13% per year, so it delivers high excess returns beyond market exposure.

## 2.3 Non-linear Market Exposure
Run the following regression on LTCM’s net excess returns:
$$
\tilde{r}_{LTCM,t} = \alpha + \beta_1 \tilde{r}_{m,t} + \beta_2 (\tilde{r}_{m,t})^2 + \epsilon_t
$$

### (a) Report $ \beta_1 $, $ \beta_2 $, and the $ R^2 $ stat.

### (b) Does the quadratic market factor increase the overall LTCM variation explained by the market?

### (c) Does LTCM’s market exposure behave as if it is long market options or short market options?

### (d) Is LTCM positively or negatively exposed to market volatility?


In [10]:
excess_spy['SPY_squared'] = excess_spy['SPY']**2

In [12]:
ut.time_series_regression_annualized(excess_returns, excess_spy, intercept=True, annual_factor=12)['df']

,Alpha,SPY Beta,SPY_squared Beta,Treynor Ratio,IR,R^2
Gross Return,0.2424,0.2198,-2.5868,-0.1023,1.8051,0.0285
Net Return,0.1550,0.1669,-1.9267,-0.0883,1.4044,0.0243


- b). The quadratic factor imprroves the $R^2$ marginally, this is likely because we're just adding more features to the regression, so $R^2$ must increase. So it doesn't help much.
The huge negative beta on SPY squared is a feature of the factor. The monthly returns are small, thus the squared returns are even smaller, thus the beta has to be larger in magnitude to fit these small returns properly.
- c). Since the beta to SPY squared returns is negative, LTCM's market exposure behaves as if it were short the market options. The beta to SPY can be interpreted as the delta of the market option and the beta to SPY squared as the gamma to market options. Since the gamma of an option is always positive, LTCM seems to be short the positive gamma or short the market options.
BUT: this is not significant, so we can't really say anything about it.
- d). For a big monthly return, the negative beta for SPY Squared would lead to heavy underperformance of LTCM returns. Big market movements would lead to big underperformance of LTCM. Thus, LTCM seems to be taking on a negative exposure to market volatility underperforming big deviations in the market and the performance not being impacted much by small deviations.

## 2.4 Up-market or Down-market Exposure
Run the following regression on LTCM’s net excess returns:
$$
\tilde{r}_{LTCM,t} = \alpha + \beta \tilde{r}_{m,t} + \beta_u \max(\tilde{r}_{m,t} - k_1, 0) + \beta_d \max(k_2 - \tilde{r}_{m,t}, 0) + \epsilon_t
$$
where $ k_1 = 0.03 $ and $ k_2 = -0.03 $.

### (a) Report $ \beta $, $ \beta_u $, $ \beta_d $, and the $ R^2 $ stat.

### (b) Is LTCM long or short the call-like factor? And the put-like factor?

### (c) Which factor moves LTCM more, the call-like factor or the put-like factor?

### (d) Does this volatility exposure come more from being long the market’s upside, short the market’s downside, or something else?


In [16]:
excess_spy['SPY_up'] = np.maximum(excess_spy['SPY'] - 0.03, 0)
excess_spy['SPY_down'] = np.maximum(-0.03 - excess_spy['SPY'], 0)

In [18]:
ut.time_series_regression_annualized(excess_returns, excess_spy[['SPY', 'SPY_up', 'SPY_down']], intercept=True, annual_factor=12)['df']

,Alpha,SPY Beta,SPY_up Beta,SPY_down Beta,Treynor Ratio,IR,R^2
Gross Return,0.1738,0.6085,-1.0384,1.6325,0.2013,1.3185,0.0638
Net Return,0.1012,0.4666,-0.7821,1.2896,0.1595,0.9320,0.0555


- b). Long puts short calls. For more of a discussion on the dynamics of this, see TA Review 8 where we discuss this! This position is called a [*risk reversal*](https://www.investopedia.com/terms/r/riskreversal.asp).
- c). We are more long puts than we are short calls, based on the magnitude of the betas. So the put-like behavior dominates.
- d). Note: this is different from last year's solutions. We know that they are short volatility, but we **cannot conclude this** based on this regression alone, and, in reality, this regression actually implies the opposite. Namely, being long options is being long vol. So, we are long a lot more put options than call options, meaning that in a vacuum, we are long vol. However, this depends on the *strikes* of the options. So if the call-like factor is closer to ATM than the put-like factor, then we might actually be short vol, since the calls will have a higher vega than the puts. However, if the strikes are symmetric (equally OTM), then we are long vol. But; we can say that we are long downside vol, and short upside vol.

# 3. The FX Carry Trade

Find an Excel data file, **“fx carry data.xlsx”**.

## 3.1 The Static Carry Trade

Define the log return of holding the foreign currency:
$$
r_i^{t+1} = s_i^{t+1} - s_i^t + rf_i^{t,t+1}
$$
Excess log return relative to USD:
$$
\tilde{r}_i^{t+1} = s_i^{t+1} - s_i^t + rf_i^{t,t+1} - rf_\$^{t,t+1}
$$

### (a) Calculate the excess log return series $\tilde{r}_t+1$ for each foreign currency $i$. Report:
- **Mean**
- **Volatility**
- **Sharpe Ratio**

### (b) What differences do you see across currencies?


In [81]:
file_path = './../data/fx_carry_data.xlsx'

# log returns
risk_free_rate = pd.read_excel(file_path, sheet_name='risk-free rates', index_col=0)
risk_free_rate.index.name = 'Date'
risk_free_rate = np.log(1 + risk_free_rate)
display(risk_free_rate.head())

fx_rates = pd.read_excel(file_path, sheet_name='fx rates', index_col=0)
fx_rates.index.name = 'Date'
fx_rates = np.log(fx_rates)
display(fx_rates.head())

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
Date,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,0.0003
1999-02-28,0.0041,0.0046,0.0026,0.0010,0.0002
1999-03-31,0.0041,0.0044,0.0025,0.0010,0.0001
1999-04-30,0.0041,0.0044,0.0021,0.0008,0.0001
1999-05-31,0.0041,0.0044,0.0021,0.0008,0.0001


,USUK,USEU,USSZ,USJP
Date,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948


In [82]:
cur_fx_map = {'GBP1M': 'USUK', 'EUR1M': 'USEU', 'CHF1M': 'USSZ', 'JPY1M': 'USJP'}
fx_holding = pd.DataFrame()
for cur, fx in cur_fx_map.items():
    fx_holding[cur] = (fx_rates[fx] - fx_rates[fx].shift(1) + risk_free_rate[cur].shift(1) - risk_free_rate['USD1M'].shift(1)).dropna()
fx_holding

,GBP1M,EUR1M,CHF1M,JPY1M
Date,,,,
1999-02-28,-0.0257,-0.0351,-0.0259,-0.0268
1999-03-31,0.0075,-0.0187,-0.0222,-0.0016
1999-04-30,-0.0031,-0.0245,-0.0352,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0160
1999-06-30,-0.0157,-0.0128,-0.0211,-0.0045
...,...,...,...,...
2021-06-30,-0.0273,-0.0293,-0.0290,-0.0112
2021-07-31,0.0077,0.0008,0.0202,0.0121
2021-08-31,-0.0120,-0.0060,-0.0117,-0.0033


In [83]:
ut.performance_metrics_annualized(fx_holding)

,Mean,Vol,Sharpe,Min,Max,Max Drawdown
GBP1M,-0.0035,0.0863,-0.0406,-0.0949,0.0883,-0.4352
EUR1M,-0.0044,0.0947,-0.0459,-0.1037,0.0935,-0.3833
CHF1M,0.0043,0.0988,0.0437,-0.1185,0.1300,-0.3132
JPY1M,-0.0174,0.0915,-0.1903,-0.0850,0.0743,-0.4390


- An FX Carry trade on CHF produces positive mean excess returns for the sample period and thus a positive sharpe. For remaining 3 currencies, we see negative returns and higher volatility and subsequently worsening sharpe ratios.

## 3.2 Implications for UIP
### (a) Do any stats contradict the (log version) of Uncovered Interest Parity (UIP)?  

### (b) Which foreign currency offered the best Sharpe ratio for a long position?  

### (c) Any foreign currencies with negative excess return?


- a). UIP states that the mean return of these currencies positions should be zero as the change in the spot fx rate is completely explained by the changes in risk free rates. However, none of the mean returns for the currencies are 0.
- b). Over the sample, a long position in CHF would have offered positive returns but a low sharpe ratio of ~0.04.
- c). All currencies except CHF earned a negative or near zero excess returns in USD terms. JPY especially had significant negative returns during the sample periods, with increased volatilities.

## 3.3 Predicting FX
Run the regression:
$$
s_i^{t+1} - s_i^t = \alpha_i + \beta_i (rf_\$^{t,t+1} - rf_i^{t,t+1}) + \epsilon_i^{t+1}
$$

### (a) Make a table of $ \alpha_i $, $ \beta_i $, and $ R^2 $.

### (b) Discuss implications of interest-rate differentials on USD strength/weakness and FX predictability.


In [84]:
regrs = []
for cur, fx in cur_fx_map.items():
    returns = fx_rates[fx].diff().to_frame(name=fx)
    factors = (risk_free_rate['USD1M'] - risk_free_rate[cur]).shift(1).to_frame(name=cur)
    regr = ut.time_series_regression_annualized(returns, factors, intercept=True, annual_factor=12)['df']
    for col in regr.columns:
        if 'Beta' in col:
            regr.rename(columns={col: 'Beta'}, inplace=True)
    regrs.append(regr)
regr_df = pd.concat(regrs)
regr_df

,Alpha,Beta,Treynor Ratio,IR,R^2
USUK,-0.0059,0.4858,-0.0167,-0.0680,0.0004
USEU,0.0070,-1.2564,-0.0006,0.0746,0.0026
USSZ,0.0436,-1.6466,-0.0116,0.4435,0.0039
USJP,-0.0060,0.3715,0.0020,-0.0656,0.0005



## 3.4 The Dynamic Carry Trade
Using:
$$
\mathbb{E}[\tilde{r}_i^{t+1}] = \alpha + (\beta - 1)(rf_\$^{t,t+1} - rf_i^{t,t+1})
$$

### (a) Calculate how often $ \mathbb{E}[\tilde{r}_i^{t+1}] > 0 $.

### (b) Identify currencies with consistent positive or negative FX risk premium.

### (c) Explain how conditional risk premia could improve static carry trade returns.

In [87]:
expectation = pd.DataFrame()

for cur, fx in cur_fx_map.items():
    alpha = regr_df.loc[fx, 'Alpha']
    beta = regr_df.loc[fx, 'Beta']
    expectation[cur] = alpha / 12 + (beta - 1) * (risk_free_rate['USD1M'] - risk_free_rate[cur]) # annualized
(expectation > 0).sum()

GBP1M     65
EUR1M    137
CHF1M    173
JPY1M      0
dtype: int64

- b). CHF displays the highest consistency in producing positive FX risk premium followed by EUR. On the other hand JPY has a negative FX risk premium during all months in the sample. GBP also has negative premiums ~75% of the months in the sample period.
- c). Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1. Since from 3.4.a JPY returns seem to be away from the expected value of 0, an improvement in the carry trade would be to short the JPY i.e. borrow at the JPY risk-free rate to invest in the USD risk-free rate. With our forecast of the USD strengthening relative to the JPY, we could be potentially getting a positive risk premia from this carry trade.